In [30]:
import pandas as pd
import pypsa

### Creating a one bus for the Netherlands

In [38]:
network = pypsa.Network()
hours_in_2023 = pd.date_range('2017-01-01 00:00Z',
                              '2017-12-31 23:00Z',
                              freq='h')

network.set_snapshots(hours_in_2023.values)

network.add("Bus",
            "electricity bus")

network.snapshots

DatetimeIndex(['2017-01-01 00:00:00', '2017-01-01 01:00:00',
               '2017-01-01 02:00:00', '2017-01-01 03:00:00',
               '2017-01-01 04:00:00', '2017-01-01 05:00:00',
               '2017-01-01 06:00:00', '2017-01-01 07:00:00',
               '2017-01-01 08:00:00', '2017-01-01 09:00:00',
               ...
               '2017-12-31 14:00:00', '2017-12-31 15:00:00',
               '2017-12-31 16:00:00', '2017-12-31 17:00:00',
               '2017-12-31 18:00:00', '2017-12-31 19:00:00',
               '2017-12-31 20:00:00', '2017-12-31 21:00:00',
               '2017-12-31 22:00:00', '2017-12-31 23:00:00'],
              dtype='datetime64[ns]', name='snapshot', length=8760, freq=None)

#### Getting the demand for the Netherlands and adding it to the bus

In [39]:
# Define file path
file_path = "demand_data.csv"

# Load dataset
data = pd.read_csv(file_path, sep=';', parse_dates=['utc_timestamp'], index_col='utc_timestamp')

# Rename columns (if necessary)
data.columns = [
    'cet_cest_timestamp', 'NL_load_actual_entsoe_transparency',
    'NL_load_forecast_entsoe_transparency', 'NL_solar_generation_actual',
    'NL_wind_generation_actual', 'NL_wind_offshore_generation_actual',
    'NL_wind_onshore_generation_actual'
]

# Convert index to datetime format
data.index = pd.to_datetime(data.index, utc=True)

start_date = '2017-01-01 00:00:00+00:00'
end_date = '2017-12-31 23:00:00+00:00'

data = data[start_date:end_date]
print(data.head())
country = 'NLD'

                                 cet_cest_timestamp  \
utc_timestamp                                         
2017-01-01 00:00:00+00:00  2017-01-01T01:00:00+0100   
2017-01-01 01:00:00+00:00  2017-01-01T02:00:00+0100   
2017-01-01 02:00:00+00:00  2017-01-01T03:00:00+0100   
2017-01-01 03:00:00+00:00  2017-01-01T04:00:00+0100   
2017-01-01 04:00:00+00:00  2017-01-01T05:00:00+0100   

                           NL_load_actual_entsoe_transparency  \
utc_timestamp                                                   
2017-01-01 00:00:00+00:00                             10794.0   
2017-01-01 01:00:00+00:00                             10320.0   
2017-01-01 02:00:00+00:00                              9930.0   
2017-01-01 03:00:00+00:00                              9679.0   
2017-01-01 04:00:00+00:00                              9652.0   

                           NL_load_forecast_entsoe_transparency  \
utc_timestamp                                                     
2017-01-01 00:00:00+00:0

In [40]:
# add load to the bus
network.add("Load",
            "load",
            bus="electricity bus",
            p_set=data['NL_load_actual_entsoe_transparency'].values)

Index(['load'], dtype='object')

In [41]:
network.loads_t.p_set

Load,load
snapshot,
2017-01-01 00:00:00,10794.0
2017-01-01 01:00:00,10320.0
2017-01-01 02:00:00,9930.0
2017-01-01 03:00:00,9679.0
2017-01-01 04:00:00,9652.0
...,...
2017-12-31 19:00:00,12703.0
2017-12-31 20:00:00,11961.0
2017-12-31 21:00:00,11478.0


#### Adding different generation types 

In [42]:
def annuity(n,r):
    """ Calculate the annuity factor for an asset with lifetime n years and
    discount rate  r """

    if r > 0:
        return r/(1. - 1./(1.+r)**n)
    else:
        return 1/n
    
print(annuity(25,0.07))
print(annuity(25,0.07)*425000*(1+0.03))

0.0858105172206656
37563.55391334636


In [43]:
# add the different carriers, only gas emits CO2
network.add("Carrier", "gas", co2_emissions=0.19) # in /MWh_tt_CO2h
network.add("Carrier", "coal", co2_emissions=0.3) # in /MWh_tt_CO2h
network.add("Carrier", "offshorewind")
network.add("Carrier", "onshorewind")
network.add("Carrier", "solar")

# add onshore wind generator
df_onshorewind = pd.read_csv('Data/onshore_wind_1979-2017.csv', sep=';', index_col=0)
df_onshorewind.index = pd.to_datetime(df_onshorewind.index)
CF_wind = df_onshorewind[country][[hour.strftime("%Y-%m-%dT%H:%M:%SZ") for hour in network.snapshots]]
capital_cost_onshorewind = annuity(30,0.07)*910000*(1+0.033) # in €/MW
network.add("Generator",
            "onshorewind",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="onshorewind",
            #p_nom_max=1000, # maximum capacity can be limited due to environmental constraints
            capital_cost = capital_cost_onshorewind,
            marginal_cost = 0,
            p_max_pu = CF_wind.values)

# add offshore wind generator
df_offshorewind = pd.read_csv('Data/offshore_wind_1979-2017.csv', sep=';', index_col=0)
df_offshorewind.index = pd.to_datetime(df_offshorewind.index)
CF_wind = df_offshorewind[country][[hour.strftime("%Y-%m-%dT%H:%M:%SZ") for hour in network.snapshots]]
capital_cost_offshorewind = annuity(30,0.07)*1000000*(1+0.033) # in €/MW
network.add("Generator",
            "offshorewind",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="offshorewind",
            #p_nom_max=1000, # maximum capacity can be limited due to environmental constraints
            capital_cost = capital_cost_offshorewind,
            marginal_cost = 0,
            p_max_pu = CF_wind.values)

# add solar PV generator
df_solar = pd.read_csv('Data/pv_optimal.csv', sep=';', index_col=0)
df_solar.index = pd.to_datetime(df_solar.index)
CF_solar = df_solar[country][[hour.strftime("%Y-%m-%dT%H:%M:%SZ") for hour in network.snapshots]]
capital_cost_solar = annuity(25,0.07)*425000*(1+0.03) # in €/MW
network.add("Generator",
            "solar",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="solar",
            #p_nom_max=1000, # maximum capacity can be limited due to environmental constraints
            capital_cost = capital_cost_solar,
            marginal_cost = 0,
            p_max_pu = CF_solar.values)

# add OCGT (Open Cycle Gas Turbine) generator
capital_cost_OCGT = annuity(25,0.07)*560000*(1+0.033) # in €/MW
fuel_cost = 21.6 # in €/MWh_th
efficiency = 0.39 # MWh_elec/MWh_th
marginal_cost_OCGT = fuel_cost/efficiency # in €/MWh_el
network.add("Generator",
            "OCGT",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="gas",
            #p_nom_max=1000,
            capital_cost = capital_cost_OCGT,
            marginal_cost = marginal_cost_OCGT)


# add coal power plant  generator
capital_cost_coal = annuity(30,0.07)*700000*(1+0.033) # in €/MW
fuel_cost = 30 # in €/MWh_th
efficiency = 0.42 # MWh_elec/MWh_th
marginal_cost_coal = fuel_cost/efficiency # in €/MWh_el
network.add("Generator",
            "coal",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="coal",
            #p_nom_max=1000,
            capital_cost = capital_cost_coal,
            marginal_cost = marginal_cost_coal)

Index(['coal'], dtype='object')

In [44]:
network.generators_t.p_max_pu

Generator,offshorewind,onshorewind,solar
snapshot,,,
2017-01-01 00:00:00,0.691,0.350,0.0
2017-01-01 01:00:00,0.732,0.377,0.0
2017-01-01 02:00:00,0.743,0.385,0.0
2017-01-01 03:00:00,0.748,0.400,0.0
2017-01-01 04:00:00,0.734,0.406,0.0
...,...,...,...
2017-12-31 19:00:00,0.711,0.468,0.0
2017-12-31 20:00:00,0.809,0.595,0.0
2017-12-31 21:00:00,0.877,0.684,0.0


In [45]:
network.optimize(solver_name='gurobi')

Index(['electricity bus'], dtype='object', name='Bus')
Index(['electricity bus'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 2/2 [00:00<00:00,  5.71it/s]
INFO:linopy.io: Writing time: 2.58s


Set parameter Username


INFO:gurobipy:Set parameter Username


GurobiError: License expired 2025-01-29